In [ ]:
from keras.datasets import mnist
import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import time
import random

In [ ]:
from keras import layers
from keras import models
from keras.layers import (
    BatchNormalization, SeparableConv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
)

In [ ]:
def showImage(im):
    im = np.array(im)
    if(len(im.shape)==1):
        plt.imshow([im],cmap = plt.cm.binary)
        return
    plt.imshow(im,cmap = plt.cm.binary)
    plt.show()

In [ ]:
train_images_labels = []

path = "/Users/seanyao/cs/ML Keras/Pneumonia CNN/chest_xray/train/"
pneumonia_train = list(map(lambda x: (path + "/PNEUMONIA/" + x, 1), next(os.walk(path + "PNEUMONIA"))[2]))
normal_train =  list(map(lambda x: (path + "/NORMAL/" + x, 0), next(os.walk(path + "NORMAL"))[2]))

train_images_labels.extend(normal_train)
train_images_labels.extend(pneumonia_train)
random.shuffle(train_images_labels)



test_images_labels = []
path = "/Users/seanyao/cs/ML Keras/Pneumonia CNN/chest_xray/test/"
pneumonia_test = list(map(lambda x: (path + "/PNEUMONIA/" + x, 1), next(os.walk(path + "PNEUMONIA"))[2]))
normal_test =  list(map(lambda x: (path + "/NORMAL/" + x, 0), next(os.walk(path + "NORMAL"))[2]))

test_images_labels.extend(normal_test)
test_images_labels.extend(pneumonia_test)
random.shuffle(test_images_labels)

val_images_labels = []
path = "/Users/seanyao/cs/ML Keras/Pneumonia CNN/chest_xray/val/"
pneumonia_val = list(map(lambda x: (path + "/PNEUMONIA/" + x, 1), next(os.walk(path + "PNEUMONIA"))[2]))
normal_val =  list(map(lambda x: (path + "/NORMAL/" + x, 0), next(os.walk(path + "NORMAL"))[2]))

val_images_labels.extend(normal_val)
val_images_labels.extend(pneumonia_val)
random.shuffle(val_images_labels)



In [ ]:
train_images = [np.array(Image.open(t[0])) for t in train_images_labels]
train_labels = [t[1] for t in train_images_labels]
test_images = [np.array(Image.open(t[0])) for t in test_images_labels]
test_labels = [t[1] for t in test_images_labels]
val_images = [np.array(Image.open(t[0])) for t in val_images_labels]
val_labels = [t[1] for t in val_images_labels]


In [ ]:
def resize(im, w,h):
    a = np.array(im)
    width,height = im.shape
    t_width = w
    t_height = h
    # a=Image.new(mode="P", size=(128,128))
    if(width*(w/h)>=height):
        t_width = int(height*(w/h))
        a = im[(width-t_width)//2:width - (width-t_width)//2,0:height]
    elif(width*(w/h)<height):
        t_height = int(width*(w/h))
        a = im[0:width, (height-t_height)//2:height - (height-t_height)//2]
        # print(a[0:100, 0:100].size)
    a = Image.fromarray(np.uint8(a))
    return np.array(a.resize((w,h)))

def flipImHor(im):
    return im[:, ::-1]


def addNoise(im, level, amp):
    noise = (np.random.rand(level)-0.5)*amp
    w,h =im.shape
    im = np.array(im)
    for i in noise:
        im[np.random.randint(0,w)][np.random.randint(0,h)]+=i
    return np.array(Image.fromarray(np.uint8(im)))

# for i in range(len(train_images)):
#     w,h,*n = train_images[i].shape
#     if(w>h):
#         print(i)
showImage(train_images[0])
# showImage(flipImHor(train_images[0]))
showImage(addNoise(train_images[0], 10000, 20))


In [ ]:
#preprocessing to reize all image to 512x512, add images that are flipped, noise added

targetWidth, targetHeight = 256,256
targetDim = (targetWidth, targetHeight)
flipped = []
for j,i in enumerate(train_images):
    if i.ndim == 3:
        train_images[j] = resize(i[:,:,1],*targetDim)
        continue
    train_images[j] = resize(i, *targetDim)
    print(j)
    flipped.append(flipImHor(train_images[j]))
    train_labels.append(train_labels[j])
for i in flipped:
    train_images.append(i)


flipped = []
for j,i in enumerate(test_images):
    if i.ndim == 3:
        test_images[j] = resize(i[:,:,1],*targetDim)
        continue
    test_images[j] = resize(i,*targetDim)
    flipped.append(flipImHor(test_images[j]))
    test_labels.append(test_labels[j])
for i in flipped:
    test_images.append(i)

flipped = []
for j,i in enumerate(val_images):
    if i.ndim == 3:
        val_images[j] = resize(i[:,:,1],*targetDim)
        continue
    val_images[j] = resize(i,*targetDim)
    flipped.append(flipImHor(val_images[j]))
    val_labels.append(val_labels[j])
for i in flipped:
    val_images.append(i)


In [ ]:
train_images = np.array(train_images).astype('float32')/255
test_images = np.array(test_images).astype('float32')/255
val_images = np.array(val_images).astype('float32')/255
from keras.utils import to_categorical
train_labels = to_categorical(np.array(train_labels))
test_labels = to_categorical(np.array(test_labels))
val_labels = to_categorical(np.array(val_labels))

In [ ]:
from keras import regularizers

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(16, (5,5), activation='relu', input_shape = (256,256,1)))
model.add(layers.Conv2D(16, (5,5), activation='relu'))
model.add(layers.MaxPool2D(3,3))
model.add(layers.Conv2D(32, (5,5), activation='relu'))
model.add(layers.Conv2D(32, (5,5), activation='relu'))
model.add(layers.MaxPool2D(3,3))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPool2D(2,2))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.Conv2D(128, (3,3), activation='relu'))

model.add(layers.Flatten())

model.add(layers.Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

model.summary()

In [ ]:
from keras.optimizers import RMSprop, Adam
opt = Adam(learning_rate=0.0001, decay=1e-5)

In [ ]:
model.compile(optimizer=opt,
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

In [ ]:
batch_sizes = [128,128]

for batch in batch_sizes:
    model.fit(train_images, train_labels, validation_data = (val_images, val_labels), epochs=1,batch_size=batch, shuffle=True)

In [ ]:
model.evaluate(test_images, test_labels)[0]